In [1]:
import random
from input.model.application import application, EApplicationType
from input.model.stream import stream, EStreamType
from input.model.task import task, ETaskType
from input import parser

tc = parser.parse_to_model("RA_example.flex_network_description", "RA_example.flex_network_description", False, False)
edge_devices = [es for es in tc.ES.values() if not es.is_prover()]
edge_devices

[[Verifier1], [Verifier2], [ES1], [ES2], [ES3], [ES4]]

In [4]:
NR_TESTCASES = 7
MAX_NR_COMM_APPS_PER_ES = 1
MAX_NR_FREE_APPS_PER_ES = 1

for i in range(NR_TESTCASES):
    A_strings = []
    for es in edge_devices:
        for j in range(random.randint(1, MAX_NR_COMM_APPS_PER_ES)):
            period = 2000
            # Create the application datastructure
            app = application(f"{es.id}_AppComm{j}", period, EApplicationType.EDGE)
            tasks = {}
            streams = []

            # Create tasks
            wcet_t1 = random.randint(1, 30)
            wcet_t2 = random.randint(1, 30)
            arrival_t1 = random.randint(1, 1000)
            arrival_t2 = arrival_t1 + 50
            
            
            t1 = task(f"t1-{app.id}", app.id, es.id, wcet_t1, period, arrival_t1, ETaskType.EDGE)
            
            
            random_es = es
            while random_es == es:
                random_es = random.choice(edge_devices)
            t2 = task(f"t2-{app.id}", app.id, random_es.id, wcet_t2, period, arrival_t2, ETaskType.EDGE)

            # Create stream
            s = stream(f"s-{app.id}", app.id, t1.src_es_id, {t2.src_es_id: None}, t1.id, {t2.id: None},
                       50+22, app.period, 1, False, 50, tc.key_length, tc.OH, EStreamType.NORMAL)
            
            astring = '<application name="{}" period="{}" type="{}">\n'.format(
                app.id, app.period, app.type.name
            )
            
            astring += "\t\t<tasks>\n"
            astring += "\t" * 3 + t1.xml_string()
            astring += "\t" * 3 + t2.xml_string()
            astring += "\t\t</tasks>\n"

            astring += "\t\t<streams>\n"
            astring += "\t" * 3 + s.xml_string()
            astring += "\t\t</streams>\n"

            astring += "\t</application>\n"
            A_strings.append(astring)
            
        for k in range(random.randint(1, MAX_NR_FREE_APPS_PER_ES)):
            period = 2000
            # Create the application datastructure
            app = application(f"{es.id}_AppFree{k}", period, EApplicationType.EDGE)
            tasks = {}
            streams = []

            # Create tasks
            wcet_t1 = random.randint(1, 50)
            arrival_t1 = random.randint(1, 1800)
            t1 = task(f"t1-{app.id}", app.id, es.id, wcet_t1, period, arrival_t1, ETaskType.EDGE)
            
            
            
            astring = '<application name="{}" period="{}" type="{}">\n'.format(
                app.id, app.period, app.type.name
            )
            
            astring += "\t\t<tasks>\n"
            astring += "\t" * 3 + t1.xml_string()
            astring += "\t\t</tasks>\n"

            astring += "\t</application>\n"
            A_strings.append(astring)
            
    # Serialize
    s = ""
    s += "<!-- mtu, overhead, mac_length in Byte. All times in us-->\n"
    s += '<NetworkDescription mtu="{}" frame_overhead="{}" key_length="{}" mac_length="{}">\n'.format(
        tc.W_f_max, tc.W_mac, tc.key_length, tc.OH
    )
    s += "\t<!-- Expected order: devices, links, applications(tasks, signals, (functionpaths)), streams, routes, schedule-->\n"



    # Applications
    s += "\n\t<!-- period&wcet is in us, size in Byte-->\n"

    for astring in A_strings:
        s += "\t" + astring + "\n"

    s += "</NetworkDescription>"
    open(f"RA_example_edge_apps_{i}.flex_network_description", "w").write(s)